## Introduction

**The Titanic challenge** on Kaggle is a competition in which the task is to predict the survival or the death of a given passenger based on a set of variables describing him such as his age, his sex, or his passenger class on the boat. 

I have recently achieved an accuracy score of **0.78708** on the public leaderboard. I have used **Logistic regression** for prediction. As I'm writing this post, I am ranked among the **top 9%** of all Kagglers.

## Importing libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Exploratory Data Analysis

In [ ]:
# reading train data
train=pd.read_csv('../input/titanic/train.csv')

# reading test data
test=pd.read_csv('../input/titanic/test.csv') 

In [ ]:
# printing first five rows of the data set
train.head()

In [ ]:
# number of rows and columns
train.shape 

In [ ]:
# column names
train.columns 

In [ ]:
# number of null values in dataset
train.isnull().sum() 

In [ ]:
train['Sex'].value_counts()

In [ ]:
# countplot
sns.countplot(x='Sex', data=train)

In [ ]:
train['Pclass'].value_counts()

In [ ]:
sns.countplot(x='Pclass', data=train)

In [ ]:
train['Embarked'].value_counts()

In [ ]:
sns.countplot(x='Embarked', data=train)

In [ ]:
train['SibSp'].value_counts()

In [ ]:
sns.countplot(x='SibSp', data=train)

Visualizing survival based on the gender.

In [ ]:
# new feature
train['Died'] = 1 - train['Survived']

In [ ]:
train.groupby('Sex').agg('sum')[['Survived', 'Died']].plot(kind='bar',
                                                           figsize=(10, 5),
                                                           stacked=True)

Plotting the same graph but with ratio instead.

In [ ]:
train.groupby('Sex').agg('mean')[['Survived', 'Died']].plot(kind='bar',
                                                            figsize=(10, 5),
                                                            stacked=True)

The Sex variable seems to be a discriminative feature. Women are more likely to survive.

Now, visualizing survival based on the fare.

In [ ]:
figure = plt.figure(figsize=(16, 7))
plt.hist([train[train['Survived'] == 1]['Fare'], train[train['Survived'] == 0]['Fare']], 
         stacked=True, bins = 50, label = ['Survived','Dead'])
plt.xlabel('Fare')
plt.ylabel('Number of passengers')
plt.legend()

Passengers with cheaper ticket fares are more likely to die. Put differently, passengers with more expensive tickets, and therefore a more important social status, seem to be rescued first.

## Feature Engineering

In [ ]:
titles = set()
for name in train['Name']:
    titles.add(name.split(',')[1].split('.')[0].strip())
print(titles)

In [ ]:
Title_Dictionary = {"Capt": "Officer","Col": "Officer","Major": "Officer","Jonkheer": "Royalty","Don": "Royalty","Sir" : "Royalty","Dr": "Officer","Rev": "Officer","the Countess":"Royalty","Mme": "Mrs","Mlle": "Miss","Ms": "Mrs","Mr" : "Mr","Mrs" : "Mrs","Miss" : "Miss","Master" : "Master","Lady" : "Royalty"}

In [ ]:
train['Title'] = train['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
train['Title'] = train.Title.map(Title_Dictionary)
train.head()

## Cleaning the  train dataset

In [ ]:
# dropping umwanted columns
df1=train.drop(['Name','Ticket','Cabin','PassengerId','Died'], axis=1)
df1.head()

In [ ]:
train.Title.value_counts()

In [ ]:
# Converting categorical feature to numeric
df1.Sex=df1.Sex.map({'female':0, 'male':1})
df1.Embarked=df1.Embarked.map({'S':0, 'C':1, 'Q':2,'nan':'NaN'})
df1.Title=df1.Title.map({'Mr':0, 'Miss':1, 'Mrs':2,'Master':3,'Officer':4,'Royalty':5})
df1.head()

In [ ]:
# median age of each sex
median_age_men=df1[df1['Sex']==1]['Age'].median()
median_age_women=df1[df1['Sex']==0]['Age'].median()

In [ ]:
# filling null values in 'Age' with respective median age
df1.loc[(df1.Age.isnull()) & (df1['Sex']==0),'Age']=median_age_women
df1.loc[(df1.Age.isnull()) & (df1['Sex']==1),'Age']=median_age_men

In [ ]:
# checking for null values
df1.isnull().sum()

Two null values in Embarked column

In [ ]:
# dropping rows with null value
df1.dropna(inplace=True)

In [ ]:
# Data is cleaned to have no null value
df1.isnull().sum()

In [ ]:
# cleaned dataset
df1.head()

## Feature Scaling

In [ ]:
df1.Age = (df1.Age-min(df1.Age))/(max(df1.Age)-min(df1.Age))
df1.Fare = (df1.Fare-min(df1.Fare))/(max(df1.Fare)-min(df1.Fare))

In [ ]:
df1.describe()

## Data Modelling

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df1.drop(['Survived'], axis=1),
    df1.Survived,
    test_size= 0.2,
    random_state=0,
    stratify=df1.Survived
)

- **Logistic Regression**

In [ ]:
# Logistic regression
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train, y_train)

from sklearn.metrics import accuracy_score

Y_pred = clf.predict(X_test)
accuracy_score(y_test, Y_pred)

## Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, Y_pred)
cm

In [ ]:
sns.heatmap(cm,annot=True)

## Cleaning test dataset

In [ ]:
# test dataset
test.head()

Different titles in the train set are:

In [ ]:
titles = set()
for name in test['Name']:
    titles.add(name.split(',')[1].split('.')[0].strip())
print(titles)

In [ ]:
test['Title'] = test['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
test['Title'] = test.Title.map(Title_Dictionary)
test.head()

In [ ]:
# dropping unwanted columns
df2=test.drop(['PassengerId','Name','Ticket','Cabin'], axis=1)

In [ ]:
# Converting categorical feature to numeric
df2.Sex=df2.Sex.map({'female':0, 'male':1})
df2.Embarked=df2.Embarked.map({'S':0, 'C':1, 'Q':2,'nan':'nan'})
df2.Title=df2.Title.map({'Mr':0, 'Miss':1, 'Mrs':2,'Master':3,'Officer':4,'Royalty':5})
df2.head()

In [ ]:
# Checking for null values
df2.isnull().sum()

In [ ]:
# median age of each sex
median_age_men2=df2[df2['Sex']==1]['Age'].median()
median_age_women2=df2[df2['Sex']==0]['Age'].median()

In [ ]:
# filling null values with respective median age
df2.loc[(df2.Age.isnull()) & (df2['Sex']==0),'Age']=median_age_women2
df2.loc[(df2.Age.isnull()) & (df2['Sex']==1),'Age']=median_age_men2

In [ ]:
# filling null values with median fare
df2['Fare']=df2['Fare'].fillna(df2['Fare'].median())

In [ ]:
df2.isnull().sum()

In [ ]:
# Null value in the title column
df2[df2.Title.isnull()]

Since the person is a woman with age 39, filling the title with 2 (mapped value for the title 'Mrs')

In [ ]:
df2=df2.fillna(2)

In [ ]:
# Data is cleaned to have no null value
df2.isnull().sum()

In [ ]:
# cleaned dataset
df2.head()

In [ ]:
# feature scaling
df2.Age = (df2.Age-min(df2.Age))/(max(df2.Age)-min(df2.Age))
df2.Fare = (df2.Fare-min(df2.Fare))/(max(df2.Fare)-min(df2.Fare))

In [ ]:
# test dataset
df2.head()

## Prediction

In [ ]:
pred = clf.predict(df2)

In [ ]:
pred

In [ ]:
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": pred
    })
submission.to_csv('submission.csv', index=False)

In [ ]:
pred_df = pd.read_csv('submission.csv')

In [ ]:
# visualizing predicted values
sns.countplot(x='Survived', data=pred_df)

#### Thank You